In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [ ]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

In [ ]:
df = pd.read_csv('loan_train.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

In [ ]:
df['loan_status'].value_counts()

In [ ]:
!conda install -c anaconda seaborn -y


In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()

In [ ]:
X = Feature
X[0:5]

In [ ]:
y = df['loan_status'].values
y[0:5]

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

In [ ]:
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
mean_acc

In [ ]:
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/­ 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

In [ ]:
k = 7
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
yhat = neigh.predict(X_test)
yhat[0:5]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
loanTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
loanTree

In [ ]:
X_train03, X_test03, y_train03, y_test03 = train_test_split( X, y, test_size=0.3, random_state=3)
print ('Train set:', X_train03.shape,  y_train03.shape)
print ('Test set:', X_test03.shape,  y_test03.shape)

loanTree.fit(X_train03,y_train03)
predTree = loanTree.predict(X_test03)

print (predTree [0:5])
print (y_test [0:5])

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test03, predTree))

In [ ]:
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_similarity_score

clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)
yhat = clf.predict(X_test)

print (yhat [0:5])
print (y_test [0:5])

print("Avg F1-score: %.4f" % f1_score(y_test, yhat, average='weighted'))
print("Jaccard score: %.4f" % jaccard_similarity_score(y_test, yhat))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LR

In [ ]:
yhat = LR.predict(X_test)
print (yhat [0:10])
print (y_test [0:10])

In [ ]:
from sklearn.metrics import jaccard_similarity_score
print ("Jaccard similarity score: : %.4f" % jaccard_similarity_score(y_test, yhat))

from sklearn.metrics import log_loss
yhat_prob = LR.predict_proba(X_test)
print ("LogLoss: : %.4f" % log_loss(y_test, yhat_prob))

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()

In [ ]:
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
Feature2 = test_df[['Principal','terms','age','Gender','weekend']]
Feature2 = pd.concat([Feature2,pd.get_dummies(test_df['education'])], axis=1)
Feature2.drop(['Master or Above'], axis = 1,inplace=True)
X2 = Feature2
X2 = preprocessing.StandardScaler().fit(X2).transform(X2)
X2[0:5]
X2.shape

In [ ]:
y2 = test_df['loan_status'].values
y2[0:5]
y2.shape

In [ ]:
# k nearest neighbours
yhat_knn = neigh.predict(X2)
jacscore_knn = jaccard_similarity_score(y2, yhat_knn)
f1score_knn=f1_score(y2, yhat_knn, average='weighted')
print("Jaccard score: %.4f" % jacscore_knn)
print("Avg F1-score: %.4f" % f1score_knn)

In [ ]:
# decision tree
yhat_tree = loanTree.predict(X2)
jacscore_tree = jaccard_similarity_score(y2, yhat_tree)
f1score_tree=f1_score(y2, yhat_tree, average='weighted')
print("Jaccard score: %.4f" % jacscore_tree)
print("Avg F1-score: %.4f" % f1score_tree)

In [ ]:
#svm
yhat_svm = clf.predict(X2)
jacscore_svm = jaccard_similarity_score(y2, yhat_svm)
f1score_svm=f1_score(y2, yhat_svm, average='weighted')
print("Jaccard score: %.4f" % jacscore_svm)
print("Avg F1-score: %.4f" % f1score_svm)

In [ ]:
#LR
yhat_lr = LR.predict(X2)
jacscore_lr = jaccard_similarity_score(y2, yhat_lr)
f1score_lr=f1_score(y2, yhat_lr, average='weighted')
print("Jaccard score: %.4f" % jacscore_lr)
print("Avg F1-score: %.4f" % f1score_lr)
yhat_prob_lr = LR.predict_proba(X2)
logloss_lr = log_loss(y2, yhat_prob_lr)
print ("LogLoss: %.4f" % logloss_lr)

In [ ]:
from IPython.display import Markdown as md
md("| Algorithm | Jaccard  | F1-score  | LogLoss |\n| ------ | ------- | -------- | ------- |\n| KNN    | %.4f    |  %.4f   |    NA    |\n| Decision Tree | %.4f | %.4f | NA |\n| SVM | %.4f | %.4f | NA |\n| LogisticRegression | %.4f | %.4f | %.4f |"
   % (jacscore_knn, f1score_knn, jacscore_tree, f1score_tree, jacscore_svm, f1score_svm, jacscore_lr, f1score_lr, logloss_lr))